## Import Libraries

In [1]:
import torch
import argparse
from torch import nn
import torch.nn as nn                     # neural networks
import torch.autograd as autograd         # computation graph
from torch import optim
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.
from torch import Tensor                  # tensor node in the computation graph
from torch.utils.data import DataLoader 
from sklearn.gaussian_process import GaussianProcessRegressor as GPR
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
import numpy as np
from scipy import io
import matplotlib.pyplot as plt
import argparse
import os
import scipy.io
from scipy.interpolate import griddata
import sys
from collections import OrderedDict
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
import time
from pyDOE import lhs         #Latin Hypercube Sampling
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name()) 

cpu


## Data Preparation

In [2]:
alpha = 0.01/np.pi
data = scipy.io.loadmat('data/burgers_shock.mat')
t = torch.tensor(data['t'], dtype = torch.float32) 
x = torch.tensor(data['x'], dtype = torch.float32) 
Exact_u = torch.tensor(data['usol'], dtype = torch.float32) 
X, T = np.meshgrid(x,t)
X_star = torch.tensor(np.hstack((X.flatten()[:,None], T.flatten()[:,None])), dtype = torch.float32)
u_star = torch.flatten(torch.transpose(Exact_u,0,1))

print(t.shape)

torch.Size([100, 1])


## Define Functions

In [11]:
def InitialCondition(N0, LB, UB, InError = 0.):  # for each sub-domain, select the points at t = 0
    x = torch.tensor([])
    if (type(LB) != type(x)):
        LB = torch.tensor(LB).cpu()
    else:
        LB = LB.cpu()
    if (type(UB) != type(x)):
        UB = torch.tensor(UB).cpu()
    else:
        UB = UB.cpu()
    indices = (X_star[:,0] >= LB) & (X_star[:,0] < UB) & (X_star[:,1] == 0.)   # select all the points between LB and UB(right)
    XT0 = X_star[indices]
    u0 = u_star[indices]
    indices = np.random.choice(XT0.shape[0], N0, replace=False)
    XT0 = XT0[indices]
    u0 = u0[indices]
     
    
    return XT0, u0, u0 + InError*torch.randn_like(u0)

def BoundaryCondition(Nb, LB, UB):  # interface and boundary along time for each sub-domain
    x = torch.tensor([])
    if (type(LB) != type(x)):
        LB = torch.tensor(LB).cpu()
    else:
        LB = LB.cpu()
    if (type(UB) != type(x)):
        UB = torch.tensor(UB).cpu()
    else:
        UB = UB.cpu()
    tb_indices = np.random.choice(t.shape[0], Nb, replace=False)
    
    tb = t[tb_indices]
    XTL = torch.cat(( LB*torch.ones((Nb,1)), tb.reshape(-1,1)), dim = 1)
    XTL.requires_grad_()
    XTU = torch.cat(( UB*torch.ones((Nb,1)), tb.reshape(-1,1)), dim = 1)
    XTU.requires_grad_()
    
     
    
    return  XTL, XTU

def MeshGrid(LBs, UBs, Nf,lhs_option=True):  # generate mesh (data) in each sub-domain
    x = torch.tensor([])
 
    if (type(LBs) != type(x)):
        LBs = torch.tensor(LBs).cpu()
    else:
        LBs = LBs.cpu()
    if (type(UBs) != type(x)):
        UBs = torch.tensor(UBs).cpu()
    else:
        UBs = UBs.cpu()
        
    if lhs_option == True:
        XTGrid = LBs.detach().numpy() + (UBs.detach().numpy()-LBs.detach().numpy())*lhs(2, Nf)
        print(XTGrid)
        
    else:
        n = round(Nf**0.5)
        XGrid, TGrid = np.meshgrid(np.arange(LBs[0], UBs[0], (UBs[0] - LBs[0])/(n-1)), 
                               np.arange(LBs[1], UBs[1], (UBs[1] - LBs[1])/(n-1))) # generate mesh not as in data
        XTGrid = np.append(XGrid.reshape(1,-1), TGrid.reshape(1,-1), axis = 0).T

    
   
    
    return XTGrid



def InterfaceCondition(x_value, LB, UB, Ni): # interface mesh (grids) along time randomly select Ni values
    x = torch.tensor([])
    if (type(LB) != type(x)):
        LB = torch.tensor(LB).cpu()
    else:
        LB = LB.cpu()
    if (type(UB) != type(x)):
        UB = torch.tensor(UB).cpu()
    else:
        UB = UB.cpu()
    TGrid = np.arange(LB, UB, (UB - LB)/(Ni*10))   # time difference
    boundary_indices = np.random.choice(TGrid.shape[0], Ni, replace = False)   
    XTGrid = np.append(x_value.cpu()*torch.tensor(np.ones((1, TGrid.shape[0]))), TGrid.reshape(1,-1), axis = 0).T
    
    return torch.tensor(XTGrid[boundary_indices], dtype = torch.float32, requires_grad=True)



In [12]:
class PINN(nn.Module):
    def __init__(self, LBs, UBs, Layers, N0, Nb, Nf, Nt, 
                 InError = 0., Activation = nn.Tanh(), 
                 model_name = "PINN.model", device = 'cpu',
                 do_smoothing = False, N0pool = 0, N01 = 1000,
                 threshold = 0.9, display_freq = 100):
        super(PINN, self).__init__()
        self.LBs = torch.tensor(LBs, dtype=torch.float32).to(device)
        self.UBs = torch.tensor(UBs, dtype=torch.float32).to(device)
        self.Layers = Layers
        self.in_dim  = Layers[0]
        self.out_dim = Layers[-1]
        self.N0 = N0
        self.Nb = Nb
        self.Nf = Nf
        self.Nt = Nt
        self.N01 = np.minimum(N0,N01)
        self.InError = InError
        self.Activation = Activation
        self.do_smoothing = do_smoothing
        self.N0pool = np.maximum(N0pool, N0)
        self.threshold = threshold
        self.XT0, self.u0_true, self.u0_err  = InitialCondition(self.N0pool, LBs[0], UBs[0], InError)
        if do_smoothing:
            self.u0, self.GP_U, self.u_selections, self.IP_U_indices = self.Smoothing(self.XT0, self.u0_err)
        else:
            self.u0, self.GP_U, self.u_selections, self.IP_U_indices = self.u0_err, None, self.N0, range(self.N0)
        self.XT0 = self.XT0.to(device)
        self.u0 = self.u0.to(device) 
        self.XTbL, self.XTbU = BoundaryCondition(self.Nb, self.LBs[0], self.UBs[0])
         
        self.XTbL = self.XTbL.to(device) 
        self.XTbU = self.XTbU.to(device)
        self.device = device
        self._nn = self.build_model()
        
        self._nn.to(self.device)
        self.Loss = torch.nn.MSELoss(reduction='mean')
        self.model_name = model_name
        self.display_freq = display_freq                        # every 100 printout my understanding
#         self.Linears = nn.ModuleList([nn.Linear(Layers[i], Layers[i+1]) for i in range(len(Layers)-1)])
    
    def build_model(self):
        Seq = nn.Sequential()
        for ii in range(len(self.Layers)-1):
            this_module = nn.Linear(self.Layers[ii], self.Layers[ii+1])
            nn.init.xavier_normal_(this_module.weight)
            Seq.add_module("Linear" + str(ii), this_module)
            if not ii == len(self.Layers)-2:
                Seq.add_module("Activation" + str(ii), self.Activation)
        return Seq

        
    
    def forward(self, x):
        x = x.to(self.device)
        x = x.reshape((-1,self.in_dim))  
#         x = 2*(x - self.LBs)/(self.UBs - self.LBs) - 1.0
        
        a = x.float()
        

        for i in range(len(self.Layers)-2):

            z = self._nn[i](a)

            a = self.Activation(z)

        out = self._nn[-1](a)


        return out
        
#         return torch.reshape(self._nn.forward(x), (-1, self.out_dim))

    def ICLoss(self):
        if self.do_smoothing and (not self.GP_U == None):
            XT0 = torch.cat( (torch.rand(self.N0,1).uniform_(self.LBs[0], self.UBs[0]),
                              self.LBs[1].cpu()*torch.ones((self.N0,1))), 
                             dim = 1 
                           ).to(torch.device('cpu'))
            u0 = torch.tensor(self.GP_U.predict(XT0[:,0].reshape(-1,1)), dtype = torch.float32 ).reshape(-1).to(self.device)
        else:
            XT0 = self.XT0
            u0  = self.u0
        
        UV0_pred = self.forward(XT0)
        u0_pred = UV0_pred[:,0].reshape(-1)
        return self.Loss(u0_pred, u0)


    def PhysicsLoss(self, XTGrid):
        XTGrid = XTGrid.to(self.device)
        uf = self.forward(XTGrid)[:,0]
        uf_x, uf_t = torch.autograd.grad(outputs=uf.to(self.device), 
                                   inputs=XTGrid, 
                                   grad_outputs=torch.ones(uf.shape).to(self.device), 
                                   create_graph = True,
                                   allow_unused=True)[0].T
        uf_xx = torch.autograd.grad(outputs=uf_x.to(self.device), 
                                   inputs=XTGrid, 
                                   grad_outputs=torch.ones(uf_x.shape).to(self.device),
                                   create_graph = True,
                                   allow_unused=True)[0][:,0]
        lossf =  self.Loss(uf_t + uf*uf_x, alpha*uf_xx)
    
        return lossf

In [26]:
class cPINN:
    def __init__(self, boundaries, t_domain, Layers, N0, Nb, Nf, Ni, Nt, #optimizer,
                 InError = 0., Activation = nn.Tanh(),
                 model_name = "cPINN.model", device = 'cpu',
                 do_smoothing = False, N0pool = 0, N01 = 1000,
                 threshold = 0.9, display_freq = 100, do_colehopf=False,
                 do_regularize = False, regmode = 'L2', regparam = 1.e-4):
        self.boundaries = torch.tensor(boundaries).to(device)
        self.tLow  = t_domain[0]
        self.tHigh = t_domain[1]
        self.Layers = Layers
        self.in_dim  = Layers[0]
        self.out_dim = Layers[-1]
        self.N0 = N0
        self.Nb = Nb
        self.Nf = Nf
        self.Ni = Ni
        self.Nt = Nt     
        self.N01 = np.minimum(N0,N01)   
        self.N0pool = np.maximum(N0pool, N0)
        self.display_freq = display_freq
        self.threshold = threshold
        self.do_smoothing = do_smoothing
        self.do_regularize = do_regularize
        self.regmode = regmode
        self.regparam = regparam
        self.InError = InError
        self.Activation = Activation
        self.device = device
        self.model_name = model_name
        self.Loss = torch.nn.MSELoss(reduction='mean')
        self.PINNs = self.build_model(boundaries, Layers, N0, Nb, Nf, Nt)
        self.do_colehopf = do_colehopf
        
        
        
        self.params = list(self.parameters())
#         self.optimizer = torch.optim.LBFGS(
#             self.params, 
#             lr=0.01, 
#             max_iter=50000, 
#             max_eval=50000, 
#             history_size=50,
#             tolerance_grad=1e-5, 
#             tolerance_change=1.0 * np.finfo(float).eps,
#             line_search_fn="strong_wolfe")       # can be "strong_wolfe"
        self.optimizer = optim.Adam(self.params, lr=1e-3)
        
        self.iter_n = 0
        
        self.XTGrids = []
     
        for ii in range(len(self.boundaries) - 1):
            LBs, UBs = [self.boundaries[ii], self.tLow], [self.boundaries[ii + 1], self.tHigh]
            XTGrid = MeshGrid(LBs, UBs, self.Nf)
            self.XTGrids.append(XTGrid)
        
        
        
        
    def build_model(self, boundaries, Layers, N0, Nb, Nf, Nt):   # collect all the models in the sub-domains
        list_PINNs = []
       
        for ii in range(len(boundaries) - 1):
            LBs, UBs = [boundaries[ii], self.tLow ], [boundaries[ii + 1], self.tHigh]
            list_PINNs.append(PINN(LBs, UBs, Layers, N0, Nb, Nf, Nt, self.InError, self.Activation, self.model_name, self.device, self.do_smoothing, self.N0pool, self.N01, self.threshold, self.display_freq))
        return list_PINNs
    
    def parameters(self):
        list_params = []
        for pinn in self.PINNs:
            list_params += list(pinn.parameters())
         
        return list_params
    
    def regLoss(self): # regularization loss
        loss = torch.tensor(0.0, dtype = torch.float32, device=self.device, requires_grad = True)
        list_params = self.parameters()
        for param in list_params:
            if self.regmode == 'L1':
                loss = loss + param.abs().sum()
            elif self.regmode == 'L2':
                loss = loss + (param**2).sum()
        return loss

    def BoundaryLoss(self):   # 0 boundary condition sounds like dirichlet boundary condition
        XTbL, XTbU = BoundaryCondition(self.Nb, self.boundaries[0], self.boundaries[-1])
        ub_l, ub_u = self.PINNs[0].forward(XTbL).to(self.device), self.PINNs[-1].forward(XTbU).to(self.device)
         
        return torch.mean(ub_l**2 + ub_u**2)
    
    
    def InterfaceLoss(self):  # interface condition if len(boundries）== 2， means no interface conditioin
        loss = torch.tensor(0.0, dtype = torch.float32, device=self.device, requires_grad = True)
        if len(self.boundaries) == 2:
            return loss
        else:
            for ii in range(len(self.boundaries)-2):
                XTI = InterfaceCondition(self.boundaries[ii+1], self.tLow, self.tHigh, self.Ni).to(self.device)
                ui_0 = self.PINNs[ii].forward(XTI).to(self.device)     # value from one side
                ui_1 = self.PINNs[ii+1].forward(XTI).to(self.device)   # value from the other side
                
                ui0_x = torch.autograd.grad(outputs=ui_0.to(self.device), 
                                            inputs=XTI, 
                                            grad_outputs=torch.ones(ui_0.shape).to(self.device), 
                                            create_graph = True,
                                            allow_unused=True)[0][:,0]
                ui1_x = torch.autograd.grad(outputs=ui_1.to(self.device), 
                                            inputs=XTI, 
                                            grad_outputs=torch.ones(ui_1.shape).to(self.device), 
                                            create_graph = True,
                                            allow_unused=True)[0][:,0]
                
                ui_0av,ui_1av = (ui_0+ui_1)/2,(ui_0+ui_1)/2 # average interface value, does not play a role in the loss function
                 
                
                loss = loss + self.Loss(ui_0av, ui_1av) + self.Loss(ui0_x, ui1_x) 
                
            return loss
    
    def Eval(self, xt):    # give the solution for any xt after training
        xt = xt.to(self.device)
        to_return = torch.zeros(xt.shape[0], 2).to(self.device)
        for ii in range(len(self.boundaries) - 1):
            indices = (xt[:,0] >= self.boundaries[ii]) & (xt[:,0] < self.boundaries[ii+1])
            this_xt = xt[indices]
            this_uv = self.PINNs[ii].forward(this_xt)
            to_return[indices] += this_uv
         
        return to_return
    
    
    def Total_Loss(self,weights=(1.0,1.0,1.0,1.0)):
        
        Training_Losses = []
        Test_Losses = []
         
        Total_ICLoss = torch.tensor(0.0, dtype = torch.float32, device=self.device, requires_grad = True)
        Total_BCLoss = torch.tensor(0.0, dtype = torch.float32, device=self.device, requires_grad = True)
        Total_PhysicsLoss = torch.tensor(0.0, dtype = torch.float32, device=self.device, requires_grad = True)
        for ii in range(len(self.boundaries) - 1):
#             LBs, UBs = [self.boundaries[ii], self.tLow], [self.boundaries[ii + 1], self.tHigh]
#             XTGrid_1 = MeshGrid(LBs, UBs, self.Nf)
            XTGrid = torch.tensor(self.XTGrids[ii], device=self.device, requires_grad=True)
             
            Total_ICLoss = Total_ICLoss + self.PINNs[ii].ICLoss()
            Total_PhysicsLoss = Total_PhysicsLoss + self.PINNs[ii].PhysicsLoss(XTGrid)

        Total_CHLoss = torch.tensor(0.0, dtype = torch.float32, device=self.device, requires_grad = True)
        Total_BCLoss = self.BoundaryLoss()
        InterfaceLoss = self.InterfaceLoss()
        Total_Loss = weights[0]*Total_ICLoss + weights[1]*Total_BCLoss\
                    + weights[2]*Total_PhysicsLoss + weights[3]*InterfaceLoss + 1.0*Total_CHLoss
        if self.do_regularize:
            RegLoss =  self.regparam * self.regLoss()
            Total_Loss = Total_Loss + RegLoss
        return Total_Loss
    
    
    def closure(self,weights=(1.0,1.0,1.0,1.0)):
        self.optimizer.zero_grad()
        loss = self.Total_Loss(weights=(1.0,1.0,1.0,1.0))
        loss.backward()
        
        self.iter_n+=1
        
        Training_Losses = []
        Test_Losses = []
         
        Total_ICLoss = torch.tensor(0.0, dtype = torch.float32, device=self.device, requires_grad = True)
        Total_BCLoss = torch.tensor(0.0, dtype = torch.float32, device=self.device, requires_grad = True)
        Total_PhysicsLoss = torch.tensor(0.0, dtype = torch.float32, device=self.device, requires_grad = True)
        for ii in range(len(self.boundaries) - 1):
#             LBs, UBs = [self.boundaries[ii], self.tLow], [self.boundaries[ii + 1], self.tHigh]
#             XTGrid_1 = MeshGrid(LBs, UBs, self.Nf)
            XTGrid = torch.tensor(self.XTGrids[ii], device=self.device, requires_grad=True)
            Total_ICLoss = Total_ICLoss + self.PINNs[ii].ICLoss()
            Total_PhysicsLoss = Total_PhysicsLoss + self.PINNs[ii].PhysicsLoss(XTGrid)
        if self.do_colehopf:
            LBs, UBs = [self.boundaries[0], self.tLow], [self.boundaries[-1], self.tHigh]
            XGrid, TGrid = MeshGrid(LBs, UBs, self.Nf*len(self.PINNs))
            XTGrid = torch.tensor(np.append(XGrid.reshape(1,-1), TGrid.reshape(1,-1), axis = 0).T, dtype = torch.float32, device=self.device, requires_grad=True)
            Total_CHLoss = self.ColeHopfLoss(XTGrid, XGrid.shape).to(self.device)
        else:
            Total_CHLoss = torch.tensor(0.0, dtype = torch.float32, device=self.device, requires_grad = True)
        Total_BCLoss = self.BoundaryLoss()
        InterfaceLoss = self.InterfaceLoss()
        Total_Loss = weights[0]*Total_ICLoss + weights[1]*Total_BCLoss\
                    + weights[2]*Total_PhysicsLoss + weights[3]*InterfaceLoss + 1.0*Total_CHLoss
        if self.do_regularize:
            RegLoss =  self.regparam * self.regLoss()
            Total_Loss = Total_Loss + RegLoss
       
        
        indices = np.random.choice(X_star.shape[0], self.Nt, replace=False)
        Test_XT = X_star[indices]
        Test_UV = self.Eval(Test_XT)
        u_exact = u_star[indices].reshape(-1).to(self.device)
        Test_Loss = self.Loss(u_exact, Test_UV[:,0].reshape(-1))
        Test_Losses.append(float(Test_Loss))
        Training_Losses.append(float(Total_Loss))
        
        u_predall = self.Eval(X_star).detach().numpy()
        u_pred = u_predall[:,0].reshape(-1)
        
        u_exact = u_star.to(self.device)
        
        error_u = np.linalg.norm(u_exact.detach().numpy().reshape(-1)-u_pred,2)/np.linalg.norm(u_exact.detach().numpy().reshape(-1),2)
        
        
        if self.iter_n % self.display_freq ==0:
            print("Iteration Number = {}".format(self.iter_n))
            print("\tIC Loss = {}".format(float(Total_ICLoss)))
            print("\tBC Loss = {}".format(float(Total_BCLoss)))
            print("\tPhysics Loss = {}".format(float(Total_PhysicsLoss)))
            print("\tCH Loss = {}".format(float(Total_CHLoss)))
            if self.do_regularize:
                print("\tRegularization Loss = {}".format(float(RegLoss)))
            print("\tInterface Loss = {}".format(float(InterfaceLoss)))
            print("\tTraining Loss = {}".format(float(Total_Loss)))
            print("\tTest Loss = {}".format(float(Test_Loss)))
            print("\tRelative_error = {}".format(float(error_u)))
        
        
        
        return loss
            
        
    
    def Train(self,n_iter):
        for i in range(n_iter):
            self.optimizer.step(self.closure)

        
         
    
    
    
    

In [27]:
%%time

class DummyArgumentParser:
    def add_argument(self, *args, **kwargs):
        pass

    def parse_args(self):
        args = {
            'domains': 4,
            'layers': 8,
            'nodes': 20,
            'N0': 100,
            'Nb': 100,
            'Nf': 15000,
            'Ni': 100,
            'Nt': 1000,
            'error': 0.0,
            'smooth': False,
            'N0pool': 150,
            'threshold': 1.0,
            'epochs': 15000,
            'model_name': 'PINN_model',
            'display_freq': 100,
            'do_colehopf': False,
            'regularize': False,
            'regmode': 'L2',
            'regparam': 1.e-4
        }
        return argparse.Namespace(**args)

# Set up the arguments (equivalent of argparse)
args = DummyArgumentParser().parse_args()

# Set the device (cuda if available, else cpu)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Extract the necessary arguments
NDomains = args.domains
NHiddenLayers = args.layers
boundaries = (1 / NDomains) * np.arange(0, NDomains + 1) * 2 - 1
t_domain = [0., 1.]
Layers = [2] + [args.nodes] * NHiddenLayers + [1]
N0 = int(args.N0 / NDomains)
Nf = int(args.Nf / NDomains)
N0pool = int(args.N0pool / NDomains)
Activation = nn.Tanh()

# Instantiate the cPINN class
cpinn = cPINN(boundaries=boundaries,
              t_domain=t_domain,
              Layers=Layers,
              N0=N0,
              Nb=args.Nb,
              Nf=Nf,
              Ni=args.Ni,
              Nt=args.Nt,
              InError=args.error,
              Activation=Activation,
              device=device,
              do_smoothing=args.smooth,
              N0pool=N0pool,
              threshold=args.threshold,
              do_colehopf=args.do_colehopf,
              display_freq=args.display_freq)

# Train the cPINN and obtain losses
Losses = cpinn.Train(n_iter=args.epochs)

cpu
[[-0.7218404   0.92464089]
 [-0.92864843  0.83772055]
 [-0.89417492  0.02704844]
 ...
 [-0.91167459  0.78025046]
 [-0.704892    0.22323562]
 [-0.55074753  0.22955896]]
[[-0.48794858  0.04777446]
 [-0.38472265  0.02993595]
 [-0.00955127  0.88832131]
 ...
 [-0.45826242  0.34096701]
 [-0.03050136  0.35684645]
 [-0.15670558  0.38655899]]
[[0.26870476 0.88219157]
 [0.36751332 0.23704523]
 [0.43212645 0.23845278]
 ...
 [0.43101055 0.86625556]
 [0.3082243  0.06278603]
 [0.11473873 0.86402748]]
[[0.98123355 0.8657706 ]
 [0.52116429 0.25683629]
 [0.88095859 0.39525539]
 ...
 [0.59636447 0.49989372]
 [0.66492352 0.0859124 ]
 [0.91393525 0.77894679]]
Iteration Number = 100
	IC Loss = 0.2396964281797409
	BC Loss = 0.01630297303199768
	Physics Loss = 0.014986810172284253
	CH Loss = 0.0
	Interface Loss = 0.00925294030457735
	Training Loss = 0.280239136787439
	Test Loss = 0.03426450863480568
	Relative_error = 0.2896321713924408
Iteration Number = 200
	IC Loss = 0.07820422947406769
	BC Loss = 0.00

Iteration Number = 2700
	IC Loss = 0.0017649342771619558
	BC Loss = 2.2709111362928525e-05
	Physics Loss = 0.0020212924661575098
	CH Loss = 0.0
	Interface Loss = 0.00028539152117446065
	Training Loss = 0.004094327401322706
	Test Loss = 0.0008930865442380309
	Relative_error = 0.06184425204992294
Iteration Number = 2800
	IC Loss = 0.0022077341563999653
	BC Loss = 4.321400047047064e-05
	Physics Loss = 0.0018210391800590193
	CH Loss = 0.0
	Interface Loss = 0.0011633140966296196
	Training Loss = 0.005235301499042694
	Test Loss = 0.0014634316321462393
	Relative_error = 0.06361126154661179
Iteration Number = 2900
	IC Loss = 0.0017957680393010378
	BC Loss = 4.149512278672773e-06
	Physics Loss = 0.0018246331894650194
	CH Loss = 0.0
	Interface Loss = 0.00022175330377649516
	Training Loss = 0.0038463040402737514
	Test Loss = 0.0017650254303589463
	Relative_error = 0.06136665493249893
Iteration Number = 3000
	IC Loss = 0.001691698795184493
	BC Loss = 1.961320776899811e-06
	Physics Loss = 0.0017859

Iteration Number = 5500
	IC Loss = 0.0010889213299378753
	BC Loss = 2.610649971757084e-06
	Physics Loss = 0.0010812633844249266
	CH Loss = 0.0
	Interface Loss = 0.000336756173055619
	Training Loss = 0.00250955150101039
	Test Loss = 0.001637016306631267
	Relative_error = 0.05876525864005089
Iteration Number = 5600
	IC Loss = 0.0010235462104901671
	BC Loss = 2.891430540330475e-06
	Physics Loss = 0.0011848054466231572
	CH Loss = 0.0
	Interface Loss = 0.0038818565662950277
	Training Loss = 0.006093099630756568
	Test Loss = 0.0013486607931554317
	Relative_error = 0.05864742398262024
Iteration Number = 5700
	IC Loss = 0.0011609542416408658
	BC Loss = 5.408464403444668e-06
	Physics Loss = 0.0010263762394017981
	CH Loss = 0.0
	Interface Loss = 5.263967250357382e-05
	Training Loss = 0.0022453785765676735
	Test Loss = 0.001070207916200161
	Relative_error = 0.05913292244076729
Iteration Number = 5800
	IC Loss = 0.0010294996900483966
	BC Loss = 1.069036534318002e-05
	Physics Loss = 0.0010555284518

Iteration Number = 8300
	IC Loss = 0.0012746778083965182
	BC Loss = 0.00020771440176758915
	Physics Loss = 0.0008182678559377001
	CH Loss = 0.0
	Interface Loss = 0.0005392999737523496
	Training Loss = 0.002839960025302242
	Test Loss = 0.001283925143070519
	Relative_error = 0.060074400156736374
Iteration Number = 8400
	IC Loss = 0.0008536511450074613
	BC Loss = 9.309453616879182e-07
	Physics Loss = 0.0007632121334347605
	CH Loss = 0.0
	Interface Loss = 0.00021501404989976436
	Training Loss = 0.0018328083016706362
	Test Loss = 0.0016186479479074478
	Relative_error = 0.057912517338991165
Iteration Number = 8500
	IC Loss = 0.0009965574136003852
	BC Loss = 1.4683382687508129e-05
	Physics Loss = 0.0008327982260343638
	CH Loss = 0.0
	Interface Loss = 0.0012559805763885379
	Training Loss = 0.0031000195641499965
	Test Loss = 0.0011393580352887511
	Relative_error = 0.058139167726039886
Iteration Number = 8600
	IC Loss = 0.001069319318048656
	BC Loss = 2.5597093554097228e-05
	Physics Loss = 0.000

Iteration Number = 11100
	IC Loss = 0.0008033741032704711
	BC Loss = 1.3440903785522096e-06
	Physics Loss = 0.0006295065158235659
	CH Loss = 0.0
	Interface Loss = 0.00011673149856505916
	Training Loss = 0.001550956190757249
	Test Loss = 0.0005719949258491397
	Relative_error = 0.05751683562994003
Iteration Number = 11200
	IC Loss = 0.0008208414656110108
	BC Loss = 2.441923243168276e-06
	Physics Loss = 0.0006253667561470463
	CH Loss = 0.0
	Interface Loss = 6.734900671290234e-05
	Training Loss = 0.0015159991562616012
	Test Loss = 0.0008169376524165273
	Relative_error = 0.05765770748257637
Iteration Number = 11300
	IC Loss = 0.0007273462833836675
	BC Loss = 1.6284554931189632e-06
	Physics Loss = 0.0006491324512432678
	CH Loss = 0.0
	Interface Loss = 3.593602014007047e-05
	Training Loss = 0.0014140432304963818
	Test Loss = 0.0009142511989921331
	Relative_error = 0.05719428136944771
Iteration Number = 11400
	IC Loss = 0.0008639124571345747
	BC Loss = 1.1758409073081566e-06
	Physics Loss = 0.

Iteration Number = 13900
	IC Loss = 0.0006389347254298627
	BC Loss = 1.8514858766138786e-06
	Physics Loss = 0.0005673116734276144
	CH Loss = 0.0
	Interface Loss = 9.498972758592572e-06
	Training Loss = 0.0012175968408944054
	Test Loss = 0.0020265262573957443
	Relative_error = 0.05678273364901543
Iteration Number = 14000
	IC Loss = 0.0007247551111504436
	BC Loss = 2.8985243716306286e-06
	Physics Loss = 0.0005221564575435595
	CH Loss = 0.0
	Interface Loss = 0.00043524501961655915
	Training Loss = 0.0016850550969934094
	Test Loss = 0.0011673885164782405
	Relative_error = 0.05729103460907936
Iteration Number = 14100
	IC Loss = 0.0006122949416749179
	BC Loss = 3.681127964227926e-06
	Physics Loss = 0.0005595541864775282
	CH Loss = 0.0
	Interface Loss = 2.5713212380651385e-05
	Training Loss = 0.001201243451216926
	Test Loss = 0.0014539025723934174
	Relative_error = 0.05669590085744858
Iteration Number = 14200
	IC Loss = 0.0006144735962152481
	BC Loss = 1.864343175839167e-06
	Physics Loss = 0.